In [ ]:
import os
from IPython.display import display, Image
import ipywidgets as widgets
from IPython.display import display, clear_output

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage, ChatMessage

In [ ]:
from langgraph.graph import MessagesState, StateGraph, END, START
from typing import Annotated, Sequence, TypedDict, Union

In [ ]:
api_key_input = widgets.Text(placeholder='Enter OpenAI API key', description='Key:')
key_submit = widgets.Button(description='Submit Key')
output = widgets.Output()

def on_key_submit(_):
    with output:
        os.environ["OPENAI_API_KEY"] = api_key_input.value
        print("API key set successfully!")
        display_key_form.layout.display = 'none'  # Hide after submission

key_submit.on_click(on_key_submit)
display_key_form = widgets.VBox([api_key_input, key_submit, output])
display(display_key_form)

In [ ]:
# System prompts as messages
reflection_system_message = """
Your goal is to create a reflection exercise that can help the user reflect on the content of the lesson provided, particularly in relation to their current life and challenges.

IMPORTANT: If there are previous Reflections in the conversation history, make sure to create a meaningfully different reflection that focuses on a different aspect of the lesson. Review the previous reflections and ensure there is no repetition.
IMPORTANT: Make sure that the reflection exercise you create deeply resonate with the provided Lesson input.

**Output Requirements**:

- Must be NEW and meaningfully DIFFERENT from previous reflection(s), while being related to the lesson content. (If there are previous Reflections available in history)
- DO NOT, UNDER ANY CIRCUMSTANCES, CREATE ANYTHING ELSE OTHER THAN A REFLECTION.
- The Reflection you produce, should be something the reader can complete in the next 5 mins. It should feel light and approachable—something the user can do effortlessly, even with their eyes closed, such as a CBT reframing exercise or a reflection that helps them consider their lives from a new point of view, or remember the last time they had something happen to them (related to the content of the lesson).
- Keep this section short, with maximum 1-2 "Think about this" lines. Avoid overwhelming the user with too many steps or details.
- Use concrete examples, when necessary, but keep everything brief and actionable (e.g. if you say ‘practice self-compassion’ provide an example of what this means and how it is done; if you say ‘define your values’, provide an example of what values are, etc.). This does not need to be a new line but just a brief example in parentheses. This is also explained in examples at the end. [VERY IMPORTANT]
- The user may choose to write down their personal takeaways after completing the reflection
- Always include a relevant title and go directly into the text, do not include any headers after the title to start the text (e.g., “Reflection Exercise:”)
- This reflection will be followed by a few more sections (quick win/ plan/ act) which are geared towards driving the user to implement changes. So in this section, you do not need to drive the user to take any actions yet. This is simply the beginning of their journey.
- Stick to a similar text length and format as in the example below

## **Examples of Output Template:**

- Below are two examples of good quality Reflection Exercises, to serve as output template. EXACTLY follow these examples' formats and word length. DO NOT MAKE ANY DEVIATIONS FROM IT AT ALL. [VERY IMPORTANT]
- ALSO MAKE SURE TO FORMULATE YOUR COMPLETE RESPONSE IN UNDER 150 WORDS. NEVER GO ABOVE THAT. [VERY IMPORTANT]

**Example 1:**

** This below example has examples in paranthesis, because they were required. **

---

**Title:** Embrace Stress as Your Guide

Gently close your eyes and recall a recent moment when stress crept in. Instead of viewing it as a foe, consider it a signal pointing to something significant in your life. Ask yourself:

- What core value or goal is this stress highlighting? (For instance, stress over a presentation might reflect your dedication to professional growth.)
- How can you break this challenge into smaller, actionable steps? (If overwhelmed by a task, start by outlining the main steps.)

Allow these questions to settle in your mind. Notice any thoughts or insights that emerge. If something resonates, consider writing it down to capture your reflections.

---

**Example 2:**

** This example does not contain examples in paranthesis, because they were not required. **

---

**Title:** Rediscover the Gift of Stillness

Close your eyes and think back to the last time you felt bored—a moment when there was nothing to do, and you reached for your phone or a distraction. Instead of labeling it as wasted time, imagine revisiting that moment with curiosity.

Ask yourself:

- What thoughts or emotions might have emerged if I had stayed present with the stillness instead of seeking distraction?
- Could that moment have sparked a new idea, clarity about something in my life, or a deeper understanding of myself?

Let these questions linger in your mind for a minute or two, without judgment. Notice if a specific thought, emotion, or insight comes up. If something stands out, consider jotting it down to capture the essence of this reflection.

---""".strip()

quick_win_system_message = """
We are trying to help the user achieve a short, tangible win. Your goal is to create a short, quick task that they can do to help them improve what they are struggling on (the goal is to help them feel better), using the lesson and lesson umbrella above as the topic. Please also take any context from their response from the reflection question to make it sequential.

IMPORTANT: If there are previous Quick Wins in the conversation history, make sure to create a meaningfully different reflection that focuses on a different aspect of the lesson. Review the previous Quick Wins and ensure there is no repetition.
IMPORTANT: Build upon what is laid out in the last created Reflection in chat history. And create a Quick Win related to that.

**Output Requirements**:

- Must be NEW and meaningfully DIFFERENT from previous Quick Wins(s), while being related to lesson content and latest Reflection. (If there are previous Quick Wins available in history)
- DO NOT, UNDER ANY CIRCUMSTANCES, CREATE ANYTHING ELSE OTHER THAN A QUICK WIN.
- Make sure the Quick Win is very relevant to the original Lesson input from user, and builds upon the most recent Reflection.
- It’s key to suggest something they can actively do in that moment, without much effort (i.e. meditate for 5 min, take a short 5-10min walk, do a short breathing exercise, set aside workout clothes for the next day, throw out the sugary drinks in their fridge, etc.).
- This should not involve doing an action later in the day outside of the 5-minute window.
- The goal of the quick win is to help the person perform a quick action that can ultimately help them feel better in the moment and accomplished (something like reaching out for feedback would not work since it does not help the person feel better in the moment and the action is not completed within the 5 minutes). This action should also only involve the user and not an interaction with another person.
- Ideally this is something that does NOT repeat what was done in the Reflection, and which helps the reader feel better / more hopeful / less stressed
- Journaling prompts and reflections are less preferable. Actions are more preferable.
- Always include a relevant title and go directly into the text, do not include any headers after the title to start the text (e.g., “Quick Win Task:”)

## **Example of Output Template:**

- Below is an example of good quality Quick Win, to serve as output template. EXACTLY follow this example’s format and word length. DO NOT MAKE ANY DEVIATIONS FROM IT AT ALL. [VERY IMPORTANT]
- ALSO MAKE SURE TO FORMULATE YOUR COMPLETE RESPONSE IN UNDER 110 WORDS. NEVER GO ABOVE THAT. [VERY IMPORTANT]

---

**Title:** Embrace a Moment of Stillness

Set a timer for 5 minutes. During this time, sit quietly and do nothing—no phone, no distractions. Instead, focus on observing your surroundings. Notice the sounds, the light, or even the sensation of your breath.

If your mind starts to wander, gently bring it back by asking yourself:

- *What small detail can I notice right now that I might have overlooked before?*

By simply being present, you're giving your mind the chance to recharge, fostering a sense of calm and openness to new insights.

---
""".strip()

plan_system_message = """
Your goal is to help the person create a plan, which will continue to build on their reflection and the learnings from the lesson content, and will help them set an intention and integrate the content of the lesson into their day.

IMPORTANT: If there are previous Plans in the conversation history, make sure to create a meaningfully different Plan that focuses on a different aspect of the lesson. Review the previous Plans and ensure there is no repetition.
IMPORTANT: Make sure that the plan is related to the last created Reflection and Quick Win, and builds upon them and helps the user integrate them in their life.

**Output Requirements**:

- Must be NEW and meaningfully DIFFERENT from previous Plan(s), while being related to lesson content and latest Reflection. (If there are previous Plans available in history)
- DO NOT, UNDER ANY CIRCUMSTANCES, CREATE ANYTHING ELSE OTHER THAN A PLAN.
- This plan should be focused on something they can do over the next day, which will help them with the lesson and lesson umbrella (i.e. scheduling time for an activity, re-allocating resources, preparing for a conversation, setting goals, defining their values, etc.).
- Please take them through a very clear planning process to create an action that they can do to help really ingrain this lesson and lesson umbrella. The goal of this plan and action should be to help them implement the lesson and improve their overall feelings of burnout.
- The planning process should be 4 steps (can combine a few steps if longer than 4).  Each step should reach “Step 1: *title of step*” and then the lesson (include the word step in the numbering).
- Put the response only in the steps (e.g., text should only follow a step, no introduction or conclusion outside of a step)
- Do not have a step for scheduling or setting time aside for the plan since we will have a separate module for that. Do not have a step for “writing it down” or committing.
- Consider that for some topics (i.e. self-compassion, processing emotions, etc.) the reader may not be able to plan to do something tactical, so this plan may be more geared towards helping them approach a situation with a different attitude, or set aside time to connect to their emotion, etc.
- It should not take the reader more than 30 minutes to go through the different steps of the plan and should be done in 1 day.
- Always include a relevant title

## **Example of Output Template:**

Below is an example of good quality Plan, to serve as output template. EXACTLY follow this example’s format and word length. DO NOT MAKE ANY DEVIATIONS FROM IT AT ALL. [VERY IMPORTANT]

---

**Title:** Transform Idle Moments into Creative Sparks

**Step 1:** Choose a Boredom-Friendly Activity

Think of one simple activity you enjoy or have been curious about that can be done anywhere—like doodling, brainstorming ideas, or observing patterns around you.

**Step 2:** Pick a Trigger for the Activity

Identify a specific situation where you often feel bored or distracted, such as waiting in line or during a lull at work. Decide that this will be your trigger to engage in the chosen activity.

**Step 3:** Gather Any Necessary Materials

If your activity requires tools, prepare them now. For instance, keep a small notebook and pen in your bag for sketching or jotting down ideas.

**Step 4:** Practice Letting Creativity Flow

The next time your trigger moment occurs, commit to the activity without judging the outcome. Let your mind wander or create freely—it’s about the process, not the result.

**Step 5:** Reflect and Adapt

At the end of the day, think back on your experience. Ask yourself: *Did this activity help me feel more engaged or inspired? How can I refine this practice to make it more enjoyable or effective tomorrow?*
---
""".strip()

In [ ]:
from langgraph.types import interrupt, Command
from langgraph.checkpoint.memory import MemorySaver
import uuid

In [ ]:
import uuid
from typing import TypedDict, List

In [ ]:
llm = ChatOpenAI(model="gpt-4o", temperature=0.7, max_retries=2)

In [ ]:
initial_message = HumanMessage(content="""
Below you'll be provided with the following fields, here are their definitions:
- Lesson Umbrella:  This is a high-level solution category to help someone solve burnout.
- Lesson Title: The title for the Lesson Content.
- Lesson Content: The content of the lesson, which is a key learning from a book or an online resource that can help the user implement and achieve the lesson umbrella above.

Now, start by creating a reflection for the following lesson umbrella and lesson, while adhering to provided guideline:
**Lesson Umbrella**: Recognize and Address Stress Before Burnout
**Lesson Title**: Complete the 'Stress Cycle' to Release Pent-Up Tension
**Lesson**: "After a stressful event, it's essential to help your body find its way back to calmness by completing the stress cycle. The stress cycle is your body’s natural way of dealing with challenges, activating tension and adrenaline to help you stay alert. But if your body doesn’t receive signals that the stress has passed, it can remain stuck in high alert, leading to burnout. Breaking or completing this cycle is essential to prevent long-term exhaustion.

This can be achieved through activities that help your body release pent-up energy, like exercise, dancing, or yoga. Social interactions—such as talking with a loved one—reassure your brain that you’re safe and supported. Creative pursuits and hobbies bring joy and meaning, while mindfulness practices, like deep breathing, help your body transition from tension to relaxation.

By prioritizing these activities, you help your body reset and manage stress more effectively over time."
""".strip())

In [ ]:
# Add this above the CoachUI class definition
LESSONS = [
    {
        "umbrella": "Recognize and Address Stress Before Burnout",
        "title": "Complete the 'Stress Cycle' to Release Pent-Up Tension",
        "content": """After a stressful event, it's essential to help your body find its way back to calmness by completing the stress cycle..."""
    },
    {
        "umbrella": "Accept that emotions and circumstances change",
        "title": "Embrace Changing Emotions",
        "content": """Mindfulness can be a powerful tool to help you handle your emotions without feeling overwhelmed. By using techniques like deep breathing and meditation, you learn to observe your feelings, such as anxiety or sadness, without judging them. When you focus on the present moment, you make room to fully experience your emotions and allow them to pass, rather than letting them control you. This approach not only reduces the intensity of your emotions but also enhances your ability to respond to challenges with calmness and thoughtfulness."""
    },
        {
        "umbrella": "Apply psychology-based strategies to build lasting positive habits",
        "title": "Track Progress and Celebrate Success",
        "content": """Keeping an eye on your progress and taking time to celebrate your wins are essential for building lasting habits. When you track your progress, it helps you stay motivated and gives you a clear view of your dedication and accomplishments.

Celebrating even small victories strengthens the habit and gives you a sense of achievement. For instance, if you want to make regular exercise a habit, logging your workouts and celebrating each milestone, like completing a week of workouts, can greatly enhance your motivation."""
    }
]

class LessonSelector:
    def __init__(self, callback):
        self.callback = callback
        self.lesson_dropdown = widgets.Dropdown(
            options=[(f"{lesson['title']} ({lesson['umbrella']})", idx) 
                    for idx, lesson in enumerate(LESSONS)],
            description='Choose a Lesson:',
            layout={'width': '90%'},
            style={'description_width': 'initial'}
        )
        self.start_button = widgets.Button(
            description="Start Session",
            layout={'width': '200px'}
        )
        self.container = widgets.VBox([
            widgets.HTML("<h3>Burnout Coach - Lesson Selector</h3>"),
            self.lesson_dropdown,
            self.start_button
        ], layout={'align_items': 'center'})
        
        self.start_button.on_click(self._start_session)
        
    def _start_session(self, b):
        clear_output()
        self.callback(LESSONS[self.lesson_dropdown.value])

In [ ]:
from langchain.schema import HumanMessage, AIMessage, ChatMessage
from langgraph.graph import StateGraph, END, START
from langgraph.checkpoint.memory import MemorySaver
from langgraph.types import Command, interrupt

class State(TypedDict):
    messages: list
    current_node: str
    feedback: str
    feedback_pending: bool  # New state flag

class CoachUI:
    def __init__(self):
        self.output_area = widgets.Output(
            layout={'border': '1px solid black', 'max_height': '500px', 'overflow': 'auto'}
        )
        self.input_box = widgets.Text(description="Input:")
        self.yes_no_buttons = widgets.ToggleButtons(
            options=['yes', 'no'],
            description='Choose:',
            disabled=False
        )
        self.submit_button = widgets.Button(description="Submit")
        
        self.feedback_container = widgets.VBox([
            widgets.HTML(value="<b>Your Feedback:</b>"),
            self.input_box,
            self.yes_no_buttons,
            self.submit_button
        ])
        
        self.container = widgets.VBox([
            self.output_area,
            self.feedback_container
        ])
        
        self.current_interrupt = None
        self.submit_button.on_click(self._handle_submit)
        self._hide_feedback_components()
        
    def display_ui(self):
        display(self.container)
        
    def _hide_feedback_components(self):
        self.input_box.layout.visibility = 'hidden'
        self.yes_no_buttons.layout.visibility = 'hidden'
        self.submit_button.layout.visibility = 'hidden'
        self.input_box.value = ''
        self.yes_no_buttons.value = None
        
    def _handle_submit(self, b):
        user_input = self.input_box.value.strip().lower() if self.current_interrupt['type'] == 'feedback' else self.yes_no_buttons.value
        self._hide_feedback_components()
        self.resume_graph(user_input)
    
    def show_input(self, interrupt_data):
        self.current_interrupt = interrupt_data
        with self.output_area:
            print(f"\n{interrupt_data['message']}")
            
        if interrupt_data['type'] == 'feedback':
            self.input_box.layout.visibility = 'visible'
        else:
            self.yes_no_buttons.layout.visibility = 'visible'
        self.submit_button.layout.visibility = 'visible'

class BurnoutCoachRunner:
    def __init__(self):
        self.main_container = widgets.Output()
        self.selector = LessonSelector(self._handle_lesson_selected)
        self.ui = CoachUI()
        self.ui.resume_graph = self.resume_graph
        self.checkpointer = MemorySaver()
        self.thread_id = None
        self.last_message_count = 0

        
        # Display initial selector
        with self.main_container:
            self.selector.container.layout.visibility = 'visible'
            display(self.selector.container)
        
        display(self.main_container)
        
    def _handle_lesson_selected(self, lesson):
        with self.main_container:
            clear_output()
            self.ui.display_ui()
            
            # Generate initial message
            initial_message = HumanMessage(content=f"""
            **Lesson Umbrella**: {lesson['umbrella']}
            **Lesson Title**: {lesson['title']}
            **Lesson**: {lesson['content']}

            Now, start by creating a reflection for this lesson:
            """)
            
            # Start session
            self.thread_id = str(uuid.uuid4())
            self.burnout_graph = self._initialize_graph()
            config = {"configurable": {"thread_id": self.thread_id}}
            initial_state = {
                "messages": [initial_message],
                "current_node": "start",
                "feedback": "",
                "feedback_pending": False
            }
            self._continue_execution(initial_state, config)


    def reflection_chain(self, state: State) -> dict:
        sys_msg = SystemMessage(content=reflection_system_message)
        response = llm.invoke([sys_msg] + state["messages"])
        response = AIMessage(content=response.content, name="Reflection")
        # print("\n=== Reflection Exercise ===")
        # print(response.content)
        return {
            "messages": state["messages"] + [response],
            "current_node": "reflection"
        }
    
    def quick_win_chain(self, state: State) -> dict:
        sys_msg = SystemMessage(content=quick_win_system_message)
        response = llm.invoke([sys_msg] + state["messages"])
        response = AIMessage(content=response.content, name="Quickwin")
        # print("\n=== Quick Win ===")
        # print(response.content)
        return {
            "messages": state["messages"] + [response],
            "current_node": "quick_win"
        }
    
    def plan_chain(self, state: State) -> dict:
        sys_msg = SystemMessage(content=plan_system_message)
        response = llm.invoke([sys_msg] + state["messages"])
        response = AIMessage(content=response.content, name="Plan")
        # print("\n=== Action Plan ===")
        # print(response.content)
        return {
            "messages": state["messages"] + [response],
            "current_node": "plan"
        }
    # Routing logic remains the same
    def route_feedback(self, state: State) -> str:
        user_input = state.get("feedback", "").lower()
        current_node = state.get("current_node", "")
        
        if user_input == "yes":
            if current_node == "reflection": return "quick_win"
            if current_node == "quick_win": return "plan"
            if current_node == "plan": return "ask_restart"
        return current_node

    def _initialize_graph(self):
        builder = StateGraph(State)
        
        # Add nodes (keep your existing node definitions)
        builder.add_node("reflection", self.reflection_chain)
        builder.add_node("quick_win", self.quick_win_chain)
        builder.add_node("plan", self.plan_chain)
        builder.add_node("get_feedback", self.get_user_feedback)
        builder.add_node("ask_restart", self.ask_restart)

        # Add edges (keep your existing edge configuration)
        builder.add_edge(START, "reflection")
        builder.add_edge("reflection", "get_feedback")
        builder.add_edge("quick_win", "get_feedback")
        builder.add_edge("plan", "get_feedback")
        
        builder.add_conditional_edges(
            "get_feedback",
            self.route_feedback,
            {"reflection": "reflection", "quick_win": "quick_win", 
             "plan": "plan", "ask_restart": "ask_restart"}
        )
        
        builder.add_conditional_edges(
            "ask_restart",
            lambda s: "reflection" if s.get("feedback") == "yes" else END,
            {"reflection": "reflection", END: END}
        )
        
        return builder.compile(checkpointer=self.checkpointer)

    def _should_display(self, msg):
        return isinstance(msg, (AIMessage, ChatMessage))

    def _update_display(self, messages):
        with self.ui.output_area:
            new_messages = [msg for msg in messages[self.last_message_count:] if self._should_display(msg)]
            for msg in new_messages:
                if isinstance(msg, AIMessage):
                    print(f"\n=== {msg.name} ===")
                    print(msg.content)
                elif isinstance(msg, ChatMessage) and msg.role == "developer":
                    print(f"\n{msg.content}")
            self.last_message_count = len(messages)

    def get_user_feedback(self, state: State) -> dict:
        # Check if feedback prompt already added
        if not state.get("feedback_pending", False):
            platform_msg = "Please provide your feedback on the above output.\nType 'yes' to proceed, or provide specific feedback to try again."
            messages = state["messages"] + [ChatMessage(content=platform_msg, role="developer", name="Inprove")]
            state = {**state, "messages": messages, "feedback_pending": True}
        
        # Get user input via interrupt
        user_input = interrupt({
            "message": platform_msg,
            "current_node": state["current_node"],
            "type": "feedback"
        })
        
        # Update state with user input
        return {
            "messages": state["messages"] + [HumanMessage(content=user_input)],
            "current_node": state["current_node"],
            "feedback": user_input,
            "feedback_pending": False
        }

    def ask_restart(self, state: State) -> dict:
        user_input = interrupt({
            "message": "Would you like to explore another aspect of the same lesson?",
            "options": ["yes", "no"],
            "type": "restart"
        })
        
        if user_input == "yes":
            new_message = HumanMessage(content="Please generate another variation focusing on a different aspect.")
            return {
                "messages": state["messages"] + [new_message],
                "current_node": "ask_restart",
                "feedback": user_input
            }
        return {"messages": state["messages"], "current_node": "end", "feedback": user_input}

    def run(self, initial_message):
        self.thread_id = str(uuid.uuid4())
        self.burnout_graph = self._initialize_graph()
        self.ui.display_ui()
        
        config = {"configurable": {"thread_id": self.thread_id}}
        initial_state = {
            "messages": [initial_message],
            "current_node": "start",
            "feedback": "",
            "feedback_pending": False
        }
        self._continue_execution(initial_state, config)
        
    def _continue_execution(self, state, config):
        stream = self.burnout_graph.stream(state, config=config)
        processed_interrupt = False
        
        try:
            for chunk in stream:
                if "__interrupt__" in chunk:
                    current_state = self.burnout_graph.get_state(config)
                    self._update_display(current_state.values["messages"])
                    self.ui.show_input(chunk["__interrupt__"][0].value)
                    processed_interrupt = True
                    break
                
                current_state = self.burnout_graph.get_state(config)
                self._update_display(current_state.values["messages"])
            
            if not processed_interrupt:
                with self.ui.output_area:
                    print("\n=== Session Complete ===")
        except StopIteration:
            pass
    
    def resume_graph(self, user_input):
        config = {"configurable": {"thread_id": self.thread_id}}
        self._continue_execution(Command(resume=user_input), config)

In [ ]:
coach = BurnoutCoachRunner()
# coach.selector.display()

In [ ]:
# for m in messages.values["messages"]:
#     m.pretty_print()

In [ ]:
# messages